# Modele generatywne dla grafów
## Jakub Binkowski
### 17, 19 czerwiec 2023

---

Poniższy zeszyt zawiera część programistyczną warsztatów. W trakcie tej części rozważymy implementacje tech wybranych modeli, a następnie przeanalizujemy otrzymane rezultaty. Skupimy się na poniższych modelach
1. VGAE [(Kipf and Welling, 2016)](https://arxiv.org/abs/1611.07308)
2. GraphVAE [(Simonovsky & Komodakis, 2018)](https://arxiv.org/abs/1802.03480)
3. DGMG [(Li et al., 2018)](https://arxiv.org/abs/1803.03324)

In [ ]:
### When working in Colab, uncomment and run the following cells ###

# !git clone https://github.com/graphml-lab-pwr/graph-generative-models-workshops-public
# !mv graph-generative-models-workshops-public/* ./
# !pip install -r requirements.txt

In [ ]:
import random
import warnings

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import seaborn as sns
import torch
from lightning_fabric import seed_everything
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from torch import Tensor, nn
from torch.nn import functional as F
from torch.optim import Adam
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCN, MLP, MeanAggregation
from torch_geometric.utils import batched_negative_sampling, dense_to_sparse
from torch_geometric.utils.convert import to_networkx
from torchmetrics.classification import BinaryAUROC
from tqdm.auto import tqdm, trange

from src.data.dataset import GeneratedDataset
from src.transforms import RandomNoiseInitialization
from src.utils.graph_vae import (
    train_graph_vae,
    visualize_graph_vae_embeddings,
    visualize_graph_vae_reconstruction,
)
from src.utils.vgae import (
    visualize_vgae_embeddings,
    visualize_vgae_reconstruction,
    visualize_vgae_training_log,
)
from src.utils.visualizations import visualize_graphs

In [ ]:
# configure plotting and device

sns.set_theme("notebook")
seed_everything(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Zbiór danych
W ramach warsztatów wykorzystamy sztuczny zbiór danych, często używany jako podstawowy benchmark do oceny jakości modelu [(You et al., 2018)](https://arxiv.org/pdf/1802.08773.pdf). Zbiór składa się z grafów, gdzie każdy graf zawiera 2 ściśle ze sobą powiązane grupy wierzchołków (*communities*), które mają małą liczbę połączeń na zewnątrz swojej grupy. Warto zaznaczyć, że jest to uproszczony zbiór, który pozwoli nam na otrzymanie wyników w relatywnie krótkim czasie. Dziś najczęściej w generowaniu grafów używa się grafów reprezentujących cząsteczki chemiczne, co często motywowane jest silną, rzeczywistą potrzebą naukowców. Pomimo tego nawet najnowsze modele, np. moele dyfuzyjne, ewaluuje się na tego typu prostych benchmarkach.

Z racji tego, że metody w dalszej części zeszytu będą oparte o Grafowe Sieci Neuronowe, wierzchołki w grafach zainicjowane będą losowym szumem Gaussowskim (poprzez dodanie transformacji `RandomNoiseInitialization`)

In [ ]:
ds = GeneratedDataset(
    "./data/datasets/community",
    "community",
    dict(num_communities=2),
    transform=RandomNoiseInitialization(dim=1),
)
ds_index = np.arange(len(ds))
train_idx, val_test_idx = train_test_split(ds_index, test_size=0.3)
val_idx, test_idx = train_test_split(val_test_idx, test_size=0.6)

train_dataset = ds[train_idx]
val_dataset = ds[val_idx]
test_dataset = ds[test_idx]

In [ ]:
num_samples = 4
fig, axes = plt.subplots(1, num_samples, figsize=(18, 4))
for ax, graph in zip(axes, train_dataset[:num_samples]):
    g = to_networkx(graph, to_undirected=True)
    nx.draw(g, ax=ax, node_size=100)
fig.suptitle("Przykładowe grafy w zbiorze danych")
plt.show()

# VGAE [(Kipf and Welling, 2016)](https://arxiv.org/abs/1611.07308)
Poniżej zaimplementujemy i przebadamy model VGAE. Model ten, jako jeden z niewielu modeli generatywnych, dostępny jest w bibliotece [Pytorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.models.VGAE.html). Jednak aby zrozumieć w pełni działanie zaimplementujemy go od podstaw. Jest to o tyle ważne, że nowsze modele często rozszerzają podejście zaproponowane w modelu VGAE, dlatego umiejętność jego implementacji oraz sprawność w modyfikacji tej metody może okazać się przydatna i fundamentalna.

<img src="assets/vgae.png" alt="Drawing" style="width: 1000px;"/>

### Ćwiczenie 1 (10 min)
Zapoznaj się z klasą `Encoder`, która będzie użyta jako warstwa kodera do modelu `VGAE`. Warstwa kodera słada się z sieci `GCN`, która transformuje nam wejściowe cechy wierzchołków (w naszym zbiorze wypełnione losowym szumem Gaussowskim) na reprezentacje stanowiące średnią i odchylenie standardowe. W pierwszej kolejności mamy wspólną sieć `GCN`, następnie jej wyjście trafia na osobne warstwy `GCN` odpowiednio predykujące wektory średni i odchyleń dla reprezentacji wierzchołków (z powodów praktycznych predykujemy logarytm wariancji zamiast odchylenia).

W ramach tego ćwiczenia zaimplementuj odpowiednie składowe klasy `Encoder`, czyli
1. `self.gcn` - początkowa sieć GNN, która pozwala nam otrzymać reprezentacje wierzchołków; aby ją zaimplementowąć użyj klasy [GCN](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.models.GCN.html?highlight=GCN#torch_geometric.nn.models.GCN) z biblioteki Pytorch Geometric:
   - wymiar wejściowy ma wartość `in_channels`
   - wymiar warstw ukrytych ma wartość `hidden_channels`
   - uwaga: musimy zarezerwować jedną warstwę na `mu` i `logvar`, zatem liczba warstw wynosi `num_layers=num_layers-1`
3. `self.mu_gcn` - pojedyncza warstwa GCN do predykcji wektora średnich `mu`:
   - wymiar wejściowy ma wartość `hidden_channles` (przyjmujemy na wejściu wyjście z `self.gcn`)
   - wymiar warstw ukrytych ma wartość `None` (używamy jednej warstwy)
   - wymiar wyjściowy ma wartość `hidden_channels`
   - liczba warstw wynosi 1
5. `self.logvar_gcn` - pojedyncza warstwa GCN do predykcji wektora średnich `logvar`:
   - wymiar wejściowy ma wartość `hidden_channles` (przyjmujemy na wejściu wyjście z `self.gcn`)
   - wymiar warstw ukrytych ma wartość `None` (używamy jednej warstwy)
   - wymiar wyjściowy ma wartość `hidden_channels`
   - liczba warstw wynosi 1

**Po zakończeniu zapoznaj się również z klasą dekodera oraz całego modelu VGAE**

In [ ]:
class Encoder(nn.Module):
    """GCN encoder with two heads for mu and logvar."""

    def __init__(self, in_channels: int, hidden_channels: int, num_layers: int):
        super().__init__()
        self.hidden_channels = hidden_channels

        # *** Fill layers implementation here ***
        self.gcn = ...
        self.mu_gcn = ...
        self.logvar_gcn = ...

    def forward(self, x: Tensor, edge_index: Tensor) -> tuple[Tensor, Tensor]:
        # forward common backbone
        h = self.gcn(x, edge_index)
        # forward to compute mu vector
        mu = self.mu_gcn(h, edge_index)
        # forward to compute logvar vector
        logvar = self.logvar_gcn(h, edge_index)
        return mu, logvar

In [ ]:
class Decoder(nn.Module):
    """MLP decoder for estimating edge probability."""

    def __init__(self, in_channels: int, hidden_channels: int, num_layers: int):
        super().__init__()
        self.mlp = MLP(
            in_channels=in_channels,
            hidden_channels=hidden_channels,
            out_channels=1,
            num_layers=num_layers,
            norm=None,
        )

    def forward(self, z: Tensor, edge_index: Tensor, sigmoid: bool = False) -> Tensor:
        # build edge_representation from nodes' representations

        z_edge = z[edge_index[0]] * z[edge_index[1]]
        # pass edge representation through decoder
        logits = self.mlp(z_edge)

        if sigmoid:
            return logits.sigmoid()

        return logits

In [ ]:
class VGAE(nn.Module):
    def __init__(self, encoder: nn.Module, decoder: nn.Module, kl_beta: float):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.kl_beta = kl_beta

    def forward(self, x: Tensor, edge_index: Tensor) -> dict[str, Tensor]:
        """Encodes input graph into latent representation vector."""
        return self.encode(x, edge_index)

    def encode(self, x: Tensor, edge_index: Tensor) -> dict[str, Tensor]:
        """Encodes input graph into latent representation vector."""
        # pass input features through encoder
        z_mu, z_logvar = self.encoder(x, edge_index)

        if self.training:
            # reparametrization trick
            std = torch.exp(0.5 * z_logvar)
            eps = torch.randn_like(std)
            z = z_mu + std * eps
        else:
            # return mean when evaluating
            z = z_mu

        # return latent representations
        return {
            "z_mu": z_mu,
            "z_logvar": z_logvar,
            "z": z,
        }

    def decode(self, z: Tensor, edge_index: Tensor) -> Tensor:
        """Decodes node representations into probabilities of edges."""
        return self.decoder(z, edge_index)

    @torch.no_grad()
    def test(
        self, z: Tensor, pos_edge_index: Tensor, batch: Tensor
    ) -> tuple[Tensor, Tensor]:
        """Sample negatives and compute proba for edges, returning proba and labels."""
        # sample negative edges
        neg_edge_index = batched_negative_sampling(pos_edge_index, batch)
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=1)

        # generate positive and negative labels for link prediction
        edge_labels = torch.ones(edge_index.shape[1], device=edge_index.device)
        edge_labels[pos_edge_index.shape[1] :] = 0

        # decode representations
        proba = self.decode(z, edge_index).sigmoid().flatten()

        return proba, edge_labels

    @torch.no_grad()
    def sample(self, num_nodes: int) -> Tensor:
        """Samples graph with a given number of nodes."""
        # sample latent representations
        z = torch.randn(num_nodes, self.encoder.hidden_channels, device=device)

        # generate lower triangular adjacency matrix
        edge_index = torch.combinations(torch.arange(num_nodes, device=device)).T

        # compute edge probability for each node combination
        edge_proba = self.decoder(z, edge_index).sigmoid().squeeze(-1)

        # keep only edges with probability higher than treshold
        gen_edge_index = edge_index[:, torch.bernoulli(edge_proba).bool()]

        return gen_edge_index

    def loss(
        self,
        z: Tensor,
        pos_edge_index: Tensor,
        z_mu: Tensor,
        z_logvar: Tensor,
        batch: Tensor,
    ) -> Tensor:
        """Computes loss as a sum of reconstruction error and KL regularization term."""
        recon_loss = self._recon_loss(z, pos_edge_index, batch)
        kl_loss = self._kl_loss(z_mu, z_logvar)

        loss = recon_loss + self.kl_beta * kl_loss

        return loss

    def _recon_loss(self, z: Tensor, pos_edge_index: Tensor, batch: Tensor) -> Tensor:
        # sample negative edges
        neg_edge_index = batched_negative_sampling(pos_edge_index, batch)
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=1)

        # generate positive and negative labels for link prediction
        edge_labels = torch.ones(edge_index.shape[1], device=edge_index.device)
        edge_labels[pos_edge_index.shape[1] :] = 0

        # decode representations
        logits = self.decode(z, edge_index).flatten()

        # compute binary cross entropy
        return F.binary_cross_entropy_with_logits(logits, edge_labels)

    def _kl_loss(self, z_mu: Tensor, z_logvar: Tensor) -> Tensor:
        return -(0.5 / len(z_mu)) * torch.mean(
            torch.sum(1 + z_logvar - z_mu**2 - z_logvar.exp(), dim=1)
        )

## Uczenie modelu
Teraz zajmiemy się uczeniem modelu VGAE. Aby wyuczyć model należy zrealizować następujące kroki
1. Budujemy model autokodera VGAE, składający się z kodera `Encoder` oraz dekodera `Decoder` (`Ćwiczenie 2`)
2. Tworzymy optymalizator `Adam` oraz odpowiednie loader'y danych
3. W kolejnych epokach i batch'ach optymalizujemy model na podstawie funkcji straty

Dla uproszczenia implementacje uczenia można wykonać przy użyciu biblioteki `PytorchLightning`, która pozwala na redukcje powtarzającego się kodu oraz alokowaniem danych na odpowiednich urządzeniach (CPU i GPU). Jednak dla celów dydaktycznych poniższa implementacja jest wykonana natywnie w `PyTorch`.

### Ćwiczenie 2 (15 min)
Przed rozpoczęciem uczenia, Twoim zadaniem będzie stworzyć instację modelu `VGAE`, co sprowadza się do następujących kroków:
1. Utwór instancje klasy `Encoder` (hiperparametry: `in_channels=1`, `hidden_channels=32`, `num_layers=3`)
2. Utwórz instancje klasy `Decoder` (hiperparametr: `in_channels=32`, `hidden_channels=32`, `num_layers=2`)
3. Utwórz instancje klasy `VGAE` (hiperparametry: `kl_beta=0.1`)

**Na koniec uruchom uczenie modelu i poczekaj do jego zakończenia**

In [ ]:
# *** Fill model instantiation here ***
encoder = ...
decoder = ...
model = ...

print(f"#parameters={sum(p.numel() for p in model.parameters())}")

In [ ]:
seed_everything(42)

# hyperparameters used for training
LR = 0.0001
BATCH_SIZE = 16
NUM_EPOCHS = 50

# use Adam optimizer
optimizer = Adam(model.parameters(), lr=LR)

# prepare dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# run training and log loss history
training_log = {
    "loss": [],
    "train_recon_auc": [],
    "val_recon_auc": [],
    "epoch": [],
}

# prepare metrics for reconstruction
train_auroc = BinaryAUROC()
val_auroc = BinaryAUROC()

model.to(device)

with trange(NUM_EPOCHS, desc="Epoch") as pbar:
    for epoch in pbar:
        batch_losses = []
        # train model
        for batch in tqdm(train_dataloader, desc="Train batches", leave=False):
            model.train()
            batch.to(device)

            # forward data and perform optimization step
            out = model.encode(batch.x, batch.edge_index)
            loss = model.loss(
                z=out["z"],
                pos_edge_index=batch.edge_index,
                z_mu=out["z_mu"],
                z_logvar=out["z_logvar"],
                batch=batch.batch,
            )
            loss.backward()
            optimizer.step()

            # compute training metrics
            model.eval()

            proba, labels = model.test(out["z"], batch.edge_index, batch.batch)
            train_auroc(proba, labels)
            batch_losses.append(loss.item())

        # log training metrics
        training_log["epoch"].append(epoch)
        epoch_loss = np.mean(batch_losses)
        training_log["loss"].append(epoch_loss)
        pbar.set_postfix({"train/loss": epoch_loss})
        training_log["train_recon_auc"].append(train_auroc.compute().item())
        train_auroc.reset()

        # validate model
        model.eval()
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Val batches", leave=False):
                batch.to(device)
                out = model(batch.x, batch.edge_index)

                # compute validation metrics
                proba, labels = model.test(out["z"], batch.edge_index, batch.batch)
                val_auroc(proba, labels)

        # log validation metrics
        training_log["val_recon_auc"].append(val_auroc.compute().item())
        val_auroc.reset()

In [ ]:
visualize_vgae_training_log(training_log)

## Rekonstrukcje grafów

In [ ]:
visualize_vgae_reconstruction(model, test_dataset, device, num_samples=6)

## Generowanie grafów
Sprawdźmy jakie grafy udało się wygenerować, oraz jak mają się one względem zbioru treningowego. Aby wygenerować grafy wykonamy następujące kroki:
1. Obliczymy rozkład wielkości grafów w zbiorze testowym
2. Wypróbkujemy rozmiary naszych grafów z otrzymanego rozkładu - musimy z góry zadać liczbę wierzchołków do wygenerowania każdego grafu

In [ ]:
def sample_vgae(model: VGAE, dataset: Dataset, num_samples: int) -> list[Data]:
    # compute graph size distribution of train set
    nodes_dist = torch.bincount(
        torch.tensor([data.num_nodes for data in train_dataset])
    )
    # sample graph sizes according to distribution of train set
    graph_sizes = torch.distributions.Categorical(nodes_dist).sample([num_samples])
    # sample graph objects (adjacency matrices)
    sampled_graphs = []
    for num_nodes in graph_sizes:
        edge_index = model.sample(num_nodes)
        g = Data(x=None, edge_index=edge_index, num_nodes=num_nodes)
        sampled_graphs.append(g)

    return sampled_graphs

In [ ]:
# compute distribution of nodes number
num_samples = 6
vgae_samples = sample_vgae(model, train_dataset, num_samples)
visualize_graphs(vgae_samples, title="Examples of graphs sampled from VGAE")

### Wizualizacje przestrzeni ukrytej

In [ ]:
visualize_vgae_embeddings(model, val_dataloader, device)
plt.show()

## Skuteczność modelu generatywnego

In [ ]:
from src.metrics import CommunityDatasetSamplingMetrics

In [ ]:
graphs_pred = sample_vgae(model, train_dataset, num_samples=len(test_dataset))

In [ ]:
metric = CommunityDatasetSamplingMetrics.from_datasets(
    train_dataset, val_dataset, test_dataset
)
metric(graphs_pred)

---
# GraphVAE [(Simonovsky & Komodakis, 2018)](https://arxiv.org/abs/1802.03480)

Teraz rozważymy ulepszenie poprzedniego modelu, czyli model `GraphVAE`. W porównaniu do `VGAE` operuje on na zagregowanych reprezentacjach całego grafu, a nie pojedynczych wierzchołków. Model ten można dostosowywać, tak aby generował cechy wierzchołków oraz krawędzi, co odgrywa kluczowe znaczenie, np. przy generowaniu molekuł. Dla uproszczenia, oraz ze względu na porównanie z modelm `VGAE`, w poniższej implementacji rozważymy tylko generowanie macierzy sąsiedztwa. Warto dodać, że pełny model `GraphVAE` powinien zawierać również element dopasowywania grafów wyjściowych. W niniejszej implementacji nie przewidujemy cech wierzchołków ani krawędzi, więc możliwa jest aproksymacja oryginalnego modelu z pominięciem dopasowywania, skróci to także czas obliczeń.

## Implementacja modelu

### Ćwiczenie 3 (15 min)
W tym ćwiczeniu zadaniem jest uzupełnienie implementacji o: (1) dwa komponenty modelu `GraphVAE`, (2) wywołanie jednego z nich w funkcji `forward`.

1. W pierwszej kolejności należy uzupełnić funkcje `__init__(self, ...)` o:
    - Warstwy agregacji wierzchołków (`self.readout`) - wykorzystaj moduł [MeanAggregation](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.aggr.MeanAggregation.html#torch_geometric.nn.aggr.MeanAggregation) z Pytorch Geoemtric
    - Warstwy dekodera (`self.decoder`) - dwuwarstwowa sieć MLP składająca się z:
      * pierwszej warstwy o wymiarze `hidden_dim, hidden_dim`
      * funkcji aktywacji `ReLU`
      * drugiej warstwy liniowej o wymiarze `hidden_dim, self.output_dim`; do implementacji MLP skorzystaj z `nn.Sequential`

2. W kolejnym kroku należy uzupełnić brakujący fragment funcji `forward`, który odpowiada za agregację reprezentacji wierzchołków do postaci reprezentacji grafu. Należy wywołać `self.readout` przekazując reprezentacje wierzchołków oraz `batch_index`. Przekazanie drugiego parametru jest konieczne, gdyż mapuje on wierzchołki na grafy, którch w batchu znajduje się 16 (określone przez `batch_size`).

**Po zakończeniu implementacji uruchom uczenie modelu i poczekaj do jego zakończenia. Następnie uruchom kolejne komórki i sprawdź jak ta wersja VAE poradziła sobie względem poprzedniej.**

In [ ]:
class GraphVAE(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        gcn_layers: int,
        max_num_nodes: int,
        kl_beta: float,
    ):
        super(GraphVAE, self).__init__()
        self.max_num_nodes = max_num_nodes
        self.hidden_dim = hidden_dim
        self.kl_beta = kl_beta

        # use for convenience
        self.triu_mask = (
            torch.triu(torch.ones(self.max_num_nodes, self.max_num_nodes)) == 1
        )

        # assume symmetric adjacency matrix (undirected graph)
        self.output_dim = max_num_nodes * (max_num_nodes + 1) // 2

        self.encoder = GCN(
            in_channels=input_dim,
            hidden_channels=hidden_dim,
            num_layers=gcn_layers,
            norm="batch_norm",
            act="relu",
        )
        self.mu_liner = nn.Linear(hidden_dim, hidden_dim)
        self.logvar_linear = nn.Linear(hidden_dim, hidden_dim)

        # *** Fill readout and deocder here ***
        self.readout = ...
        self.decoder = ...

    def forward(self, x: Tensor, edge_index: Tensor, batch_index: Tensor):
        # encoder part of VAE
        node_z = self.encoder(x, edge_index)

        # aggregate node representations into graph representation
        # *** Apply readout here ***
        graph_z = ...

        # predict mu and sigma
        z_mu = self.mu_liner(graph_z)
        z_logvar = self.logvar_linear(graph_z)

        # reparametrization trick
        z_sigma = torch.exp(0.5 * z_logvar)
        eps = torch.randn_like(z_sigma)
        z = z_mu + eps * z_sigma

        # decoder part of VAE
        edge_recon = self.decoder(z)
        return {
            "z_mu": z_mu,
            "z_logvar": z_logvar,
            "z": z,
            "edge_recon": edge_recon,
        }

    @torch.no_grad()
    def sample(self, num_samples: int) -> Tensor:
        z = torch.randn(num_samples, self.hidden_dim).to(device)
        edges = self.decoder(z).sigmoid()
        edges = torch.bernoulli(edges)
        recon_adj_lower = self._recover_adj_lower(edges)
        recon_adj_matrix = self._recover_full_adj_from_lower(recon_adj_lower)

        return recon_adj_matrix

    def loss(
        self, adj_matrix: Tensor, z_mu: Tensor, z_logvar: Tensor, edge_recon: Tensor
    ) -> dict[str, Tensor]:
        """Computes loss as a sum of adjacency reconstruction and KL regularization term."""
        adj_recon_loss = self._reconstruction_loss(adj_matrix, edge_recon)
        loss_kl = self._kl_loss(z_mu, z_logvar)
        loss = adj_recon_loss + self.kl_beta * loss_kl

        return {
            "recon": adj_recon_loss,
            "kl": loss_kl,
            "total": loss,
        }

    def _reconstruction_loss(
        self,
        adj_matrix: Tensor,
        edge_recon: Tensor,
    ) -> Tensor:
        recon_adj_lower = self._recover_adj_lower(edge_recon)
        recon_adj_matrix = self._recover_full_adj_from_lower(recon_adj_lower)

        # Here, we should obtain recon_per via graph matching procedure
        # To simplify the code and speed up computation, we omit this part

        recon_perm = recon_adj_matrix
        recon_adj = recon_perm[:, self.triu_mask]
        gold_adj = adj_matrix[:, self.triu_mask]

        return F.binary_cross_entropy_with_logits(recon_adj, gold_adj)

    def _kl_loss(self, z_mu: Tensor, z_logvar: Tensor) -> Tensor:
        return -(0.5 / len(z_mu)) * torch.mean(
            torch.sum(1 + z_logvar - z_mu**2 - z_logvar.exp(), dim=1)
        )

    def _recover_adj_lower(self, adj_matrix: Tensor):
        adj_matrix = adj_matrix.to("cpu")
        batch_size, *_ = adj_matrix.shape
        adj = torch.zeros(batch_size, self.max_num_nodes, self.max_num_nodes)
        adj[:, self.triu_mask] = adj_matrix
        return adj.to(device)

    def _recover_full_adj_from_lower(self, triu_adj: Tensor):
        diag = torch.diag_embed(torch.diagonal(triu_adj, dim1=-2, dim2=-1))
        return triu_adj + torch.transpose(triu_adj, 1, 2) - diag

## Uczenie modelu

In [ ]:
max_nodes = max(data.num_nodes for data in train_dataset)

model = GraphVAE(
    input_dim=1,
    hidden_dim=16,
    max_num_nodes=max_nodes,
    kl_beta=0.5,
    gcn_layers=2,
)

print(f"#max_nodes:{max_nodes}")
print(f"#parameters: {sum(p.numel() for p in model.parameters())}")

In [ ]:
hparams = {
    "lr": 0.001,
    "epochs": 20,
}

training_log = train_graph_vae(model, train_dataloader, val_dataloader, hparams, device)

In [ ]:
visualize_vgae_training_log(training_log)

## Rekonstrukcja grafów

In [ ]:
visualize_graph_vae_reconstruction(model, val_dataset, device, 5)

## Geneorwanie grafów

In [ ]:
model.eval()
sampled_graphs = model.sample(5)
graphs = [nx.from_numpy_array(g.cpu().numpy()) for g in sampled_graphs]

visualize_graphs(graphs, title="Examples of graphs sampled from GraphVAE")

## Wizualizacja przestrzeni ukrytej

In [ ]:
visualize_graph_vae_embeddings(model, train_dataloader, device)
plt.show()

## Skuteczność modelu generatywnego

In [ ]:
model.eval()
graphs_preds = [
    nx.from_numpy_array(adj.cpu().numpy()) for adj in model.sample(len(test_dataset))
]
metric = CommunityDatasetSamplingMetrics.from_datasets(
    train_dataset, val_dataset, test_dataset
)
metric(graphs_preds)

---
# Learning Deep Generative Models of Graphs [(Li et al., 2018)](https://arxiv.org/abs/1803.03324) (DGMG)
Kolejnym modelem, który rozważymy jest `DGMG` (nazwa pochodzi od tytułu artykułu). Ze względu na długi czas uczenia modelu wykorzystamy zbiór składający się grafów prezentujących cykle. Tego typu dane stanowią dużo łatwiejsze zadanie dla modelu niż poprzednio rozważany zbiór. W tej części wyjątkowo skorzystamy z biblioteki `DGL`, która jest alternatywą na `PyTorch Geometric`.

Poniższy kod oparty na jest na poradniku do biblioteki DGL (https://docs.dgl.ai/en/0.8.x/tutorials/models/3_generative_model/5_dgmg.html) autorstwa Mufei Li, Lingfan Yu, Zheng Zhang, z którym polecamy się zapoznać po zakończeniu warsztatów. W szczególności warto zapoznać się z implementacją poszczególnych komponentów modelu, która znajduje się w katalogu `src`.

In [ ]:
import dgl
from src.models.dgmg_components import (
    GraphEmbed,
    GraphProp,
    AddNode,
    AddEdge,
    ChooseDestAndUpdate,
)
from torch.utils.data import DataLoader

from src.models.dgmg_init import dgmg_message_weight_init, weights_init
from src.utils.dgmg_utils import animate_graph_evolution
from src.data.dgmg import CycleDataset, generate_dataset

### Model
Model DGMG składa się z kilku komponentów:
1. `add_node_agent` - sieć neuronowa, która decyduje, czy dodać kolejny wierzchołek
2. `add_edge_agent` - sieć neuronowa, która decyduje, czy dodać krawędź do nowo dodanego wierzchołka
3. `choose_dest_agent` - sieć neuronowa, która decyduje, jakie połączenia z wszystkimi wcześniej istniejącymi grafami należy utworzyć

In [ ]:
class DGMG(nn.Module):
    def __init__(self, v_max, node_hidden_size, num_prop_rounds):
        super(DGMG, self).__init__()

        # Graph configuration
        self.v_max = v_max

        # Graph embedding module
        self.graph_embed = GraphEmbed(node_hidden_size)

        # Graph propagation module
        self.graph_prop = GraphProp(num_prop_rounds, node_hidden_size)

        # Actions
        self.add_node_agent = AddNode(self.graph_embed, node_hidden_size)
        self.add_edge_agent = AddEdge(self.graph_embed, node_hidden_size)
        self.choose_dest_agent = ChooseDestAndUpdate(self.graph_prop, node_hidden_size)

        # Weight initialization
        self.init_weights()

    @property
    def action_step(self):
        old_step_count = self.step_count
        self.step_count += 1

        return old_step_count

    def forward(self, actions=None):
        # The graph we will work on
        self.g = dgl.DGLGraph()

        # If there are some features for nodes and edges,
        # zero tensors will be set for those of new nodes and edges.
        self.g.set_n_initializer(dgl.frame.zero_initializer)
        self.g.set_e_initializer(dgl.frame.zero_initializer)

        if self.training:
            return self.forward_train(actions)
        else:
            return self.forward_inference()

    def forward_train(self, actions):
        self.prepare_for_train()

        stop = self.add_node_and_update(a=actions[self.action_step])

        while not stop:
            to_add_edge = self.add_edge_or_not(a=actions[self.action_step])
            while to_add_edge:
                self.choose_dest_and_update(a=actions[self.action_step])
                to_add_edge = self.add_edge_or_not(a=actions[self.action_step])
            stop = self.add_node_and_update(a=actions[self.action_step])

        return self.get_log_prob()

    def forward_inference(self):
        stop = self.add_node_and_update()
        while (not stop) and (self.g.num_nodes() < self.v_max + 1):
            num_trials = 0
            to_add_edge = self.add_edge_or_not()
            while to_add_edge and (num_trials < self.g.num_nodes() - 1):
                self.choose_dest_and_update()
                num_trials += 1
                to_add_edge = self.add_edge_or_not()
            stop = self.add_node_and_update()

        return self.g

    def prepare_for_train(self):
        self.step_count = 0

        self.add_node_agent.prepare_training()
        self.add_edge_agent.prepare_training()
        self.choose_dest_agent.prepare_training()

    def init_weights(self):
        self.graph_embed.apply(weights_init)
        self.graph_prop.apply(weights_init)
        self.add_node_agent.apply(weights_init)
        self.add_edge_agent.apply(weights_init)
        self.choose_dest_agent.apply(weights_init)

        self.graph_prop.message_funcs.apply(dgmg_message_weight_init)

    def add_node_and_update(self, a=None):
        """Decide if to add a new node.
        If a new node should be added, update the graph."""

        return self.add_node_agent(self.g, a)

    def add_edge_or_not(self, a=None):
        """Decide if a new edge should be added."""

        return self.add_edge_agent(self.g, a)

    def choose_dest_and_update(self, a=None):
        """Choose destination and connect it to the latest node.
        Add edges for both directions and update the graph."""

        self.choose_dest_agent(self.g, a)

    def get_log_prob(self):
        return (
            torch.cat(self.add_node_agent.log_prob).sum()
            + torch.cat(self.add_edge_agent.log_prob).sum()
            + torch.cat(self.choose_dest_agent.log_prob).sum()
        )

## Uczenie modelu

Do uczenia modelu wykorzystamy zbiór grafów będących cyklami. W zakomentowanej komórce znajdziesz kod, który pozwala wcześniej używany zbiór danych przetransformować do odpowiedniej sekwencji akcji. Uczenie na nim mogłoby potrwać zbyt długo, dlatego wyuczymy model na zbiorze grafów cyklicznych.

In [ ]:
# from src.utils.dgmg_utils import tranform_dataset, collate_single

# dataset = tranform_dataset(train_dataset)

# data_loader = DataLoader(
#     dataset,
#     batch_size=1,
#     shuffle=True,
#     num_workers=0,
#     collate_fn=collate_single,
# )
# v_max = max_nodes

In [ ]:
v_min = 10
v_max = 20
n_samples = 4000

fname = "./data/datasets/cycles.pkl"
generate_dataset(
    v_min=v_min,
    v_max=v_max,
    n_samples=n_samples,
    fname=fname,
)

dataset = CycleDataset(fname)
data_loader = DataLoader(dataset, batch_size=1, num_workers=4)

In [ ]:
model = DGMG(
    v_max=v_max,
    node_hidden_size=16,
    num_prop_rounds=2,
)

print(f"#parameter: {sum(p.numel() for p in model.parameters())}")

In [ ]:
BATCH_SIZE = 1
LR = 0.0005
optimizer = Adam(model.parameters(), lr=LR)

model.train()
for epoch in trange(20, desc="Epoch"):
    batch_count = 0
    batch_loss = 0
    batch_prob = 0
    optimizer.zero_grad()

    pbar = tqdm(
        enumerate(data_loader), total=len(data_loader), leave=False, desc="Graph"
    )
    for i, data in pbar:
        log_prob = model(actions=data)
        prob = log_prob.detach().exp()

        loss = -log_prob / BATCH_SIZE
        prob_averaged = prob / BATCH_SIZE

        loss.backward()

        batch_loss += loss.item()
        batch_prob += prob_averaged.item()
        batch_count += 1

        if batch_count % BATCH_SIZE == 0:
            pbar.set_postfix({"averaged_loss": batch_loss, "averaged_prob": batch_prob})
            optimizer.step()

            batch_loss = 0
            batch_prob = 0
            optimizer.zero_grad()

        # comment the following breaks for full training
        if i > 1000:
            break
    break

## Wizualizacja procesu tworzenia grafu
Poniżej możemy obserwować jak w kolejnych krokach dołączane są kolejne wierzchołki i budowany jest graf.

In [ ]:
%matplotlib widget
animate_graph_evolution(model)

In [ ]:
# If you want, you could leverage publicly available dataset

# import torch.utils.model_zoo as model_zoo
# Download a pre-trained model state dict for generating cycles with 10-20 nodes.
# state_dict = model_zoo.load_url('https://data.dgl.ai/model/dgmg_cycles-5a0c40be.pth')
# model = DGMG(v_max=20, node_hidden_size=16, num_prop_rounds=2)
# model.load_state_dict(state_dict)
# model.eval()